In [1]:
import pandas as pd
dtypes  = {"tconst": "string[pyarrow]", "ordering": "int64", "nconst": "string", "category": "string", "job": "string", "characters": "string"}
amitabh_df = pd.read_csv("amitabh_movies.csv", dtype=dtypes)
actors_df = pd.read_csv("actors.csv")
principals_df = pd.read_csv('title.principals.tsv', sep='\t', dtype=dtypes)
principals_df = principals_df[principals_df['category'] == 'actor']

In [4]:
principals_df = principals_df[['tconst', 'nconst']].reset_index()
principals_df = principals_df[['tconst', 'nconst']]
amitabh_nconst = 'nm0000821'

In [6]:
df2 = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz' , compression='gzip', sep='\t')

/home/dsingh/.tmp/ipykernel_417125/1894314870.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz' , compression='gzip', sep='\t')


In [7]:
indian_movies = df2[(df2['region'] == 'IN') & (df2['language'] == 'hi')]

In [8]:
df3 = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz' , compression='gzip', sep='\t')

/home/dsingh/.tmp/ipykernel_417125/537892919.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz' , compression='gzip', sep='\t')


In [9]:
merged = indian_movies.merge(left_on="titleId", right_on="tconst", right=df3)

In [13]:
merged = merged[merged['titleType'] == 'movie']
merged = merged[merged['startYear'] != '\\N']
merged = merged.astype({'startYear': 'int32'})
merged = merged[merged['startYear'] > 1968]

In [28]:
# principals_df['mapped_nconst'] = principals_df['nconst'].map(mapping)
grouped = principals_df[['tconst', 'nconst']]


In [29]:
wewant = merged['tconst']
grouped = grouped.loc[grouped['tconst'].isin(wewant)]

In [32]:
grouped.reset_index(inplace=True)

In [33]:
grouped = grouped[['tconst', 'nconst']]


In [36]:
import numpy as np
def fast_grouper(df):
    keys, values = df.sort_values('tconst').values.T
    uniques, idx = np.unique(keys, True)
    arrays = np.split(values, idx[1:])
    df2 = pd.DataFrame({'tconst':uniques, 'nconst':[list(a) for a in arrays]})
    return df2

In [37]:
grouped = fast_grouper(grouped)

In [42]:
grouped.to_csv("grouped_movies_to_actors.csv", index=False)